In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
import json
import itertools
from tqdm import tqdm
from __future__ import annotations
from scipy.sparse.linalg import expm
from sklearn.linear_model import Ridge
from typing import Sequence

In [2]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [3]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [4]:
# --------------------------------------------------------------------- #
#                          Helper functions                             #
# --------------------------------------------------------------------- #
def _build_laplacian(adj: sparse.spmatrix) -> sparse.spmatrix:
    """Combinatorial Laplacian L = D - A   (sparse CSR)."""
    deg = np.asarray(adj.sum(axis=1)).ravel()
    D = sparse.diags(deg, format="csr")
    return D - adj


def _default_sequence(val: float, length: int) -> list[float]:
    """Repeat *val* 'length' times, return as list."""
    return [val] * length


# --------------------------------------------------------------------- #
#                     Diffusion-Wavelet Reservoir ESN                   #
# --------------------------------------------------------------------- #
class DWMSR3D:
    """
    DW-MSR Echo-State Network.

    Parameters
    ----------
    adj                   : scipy.sparse matrix (shape [n0,n0])
        Symmetric, unweighted or weighted adjacency of the base graph.
    num_scales            : int,   S ≥ 0   (# coarse levels)
    tau0                  : float, base diffusion time   (τ₀)
    betas                 : Sequence[float] length S,   funnel strengths β_s
    alphas                : Sequence[float] length S+1, leak per scale α_s
    input_scale           : float, scale of random W_in entries
    ridge_alpha           : float, ℓ₂ penalty in ridge read-out
    detail_features       : bool,  include Δ_s = x^{(s-1)}-x^{(s)} in Φ(x)?

    Notes
    -----
    * Reservoir size  N = (S+1) * n0
    * P_s are pre-computed once with sparse expm; they share sparsity of *adj*.
    """

    # ------------------------------------------------------------------ #
    def __init__(
        self,
        adj: sparse.spmatrix,
        num_scales: int = 2,
        tau0: float = 0.1,
        betas: Sequence[float] | None = None,
        alphas: Sequence[float] | None = None,
        input_scale: float = 0.5,
        ridge_alpha: float = 1e-6,
        detail_features: bool = True,
        seed: int = 42,
    ):
        # -------- basics -------------------------------------------------
        num_scales = len(betas) if betas is not None else num_scales
        if adj.shape[0] != adj.shape[1]:
            raise ValueError("adjacency must be square")
        if not sparse.isspmatrix(adj):
            adj = sparse.csr_matrix(adj)
        self.n0 = adj.shape[0]
        self.S = int(num_scales)
        if self.S < 0:
            raise ValueError("num_scales must be ≥ 0")

        self.N = (self.S + 1) * self.n0
        self.tau0 = float(tau0)
        self.input_scale = input_scale
        self.ridge_alpha = ridge_alpha
        self.detail_features = detail_features
        self.seed = seed

        # -------- leak & funnel parameters ------------------------------
        self.betas = list(betas) if betas is not None else _default_sequence(0.5, self.S)
        if len(self.betas) != self.S:
            raise ValueError("betas must have length S")

        self.alphas = (
            list(alphas)
            if alphas is not None
            else [0.5] + _default_sequence(1.0, self.S)  # finer quicker, coarse slow
        )
        if len(self.alphas) != self.S + 1:
            raise ValueError("alphas must have length S+1")

        # -------- internal matrices -------------------------------------
        self.Ps: list[sparse.spmatrix] = []
        self.Vs: list[np.ndarray] = []  # just β_s I, store scalars
        self._precompute_operators(adj)

        self.W_in: np.ndarray | None = None      # set in fit_readout
        self.W_out: np.ndarray | None = None

        # state block list for convenience (each block length n0)
        self.x_blocks = [np.zeros(self.n0, dtype=np.float32) for _ in range(self.S + 1)]

    # ------------------------------------------------------------------ #
    #                    Pre-computation of diffusion kernels            #
    # ------------------------------------------------------------------ #
    def _precompute_operators(self, adj: sparse.spmatrix):
        """Compute P_s and store funnel scalars β_s."""
        L = _build_laplacian(adj).tocsr()
        # largest eigenvalue bound (Gershgorin): max row sum of |L|
        lam_max = L.max(axis=1).toarray().ravel().max() + 1e-9
        if self.tau0 * (2 ** self.S) * lam_max > 50:
            print(
                "Warning: very large diffusion times may cause underflow in expm; "
                "consider reducing tau0."
            )

        for s in range(self.S + 1):
            tau_s = (2 ** s) * self.tau0
            Ps = expm((-tau_s) * L)  # still sparse CSR
            self.Ps.append(Ps)

        self.Vs = self.betas  # just scalars

    # ------------------------------------------------------------------ #
    #                            Core update                             #
    # ------------------------------------------------------------------ #
    def _single_step(self, u_t: np.ndarray):
        """
        Update all scales in causal order (fine → coarse) per eq. (1).
        """
        new_blocks = []

        # scale 0 (fine)
        z0 = self.Ps[0].dot(self.x_blocks[0]) + self.W_in.dot(u_t)
        x0_new = np.tanh(z0)
        x0_next = (1.0 - self.alphas[0]) * self.x_blocks[0] + self.alphas[0] * x0_new
        new_blocks.append(x0_next)

        # coarser scales
        for s in range(1, self.S + 1):
            z = self.Ps[s].dot(self.x_blocks[s]) + self.Vs[s - 1] * new_blocks[s - 1]
            xs_new = np.tanh(z)
            xs_next = (1.0 - self.alphas[s]) * self.x_blocks[s] + self.alphas[s] * xs_new
            new_blocks.append(xs_next)

        # commit
        self.x_blocks = new_blocks

    def reset_state(self):
        for blk in self.x_blocks:
            blk.fill(0.0)

    # ------------------------------------------------------------------ #
    #                        Read-out training                            #
    # ------------------------------------------------------------------ #
    def fit_readout(self, inputs: np.ndarray, targets: np.ndarray, discard: int = 100):
        """
        Teacher-forcing to train W_out (ridge).

        inputs  shape [T, d_in]
        targets shape [T, d_out]
        """
        T, d_in = inputs.shape
        if T <= discard + 1:
            raise ValueError("Not enough data for training")

        # random W_in
        rng = np.random.default_rng(self.seed)
        self.W_in = (
            rng.uniform(-1.0, 1.0, size=(self.n0, d_in)) * self.input_scale
        ).astype(np.float32)

        # roll through data
        self.reset_state()
        states, details = [], []
        for t in range(T):
            self._single_step(inputs[t])
            if t >= discard:
                flat_state = np.concatenate(self.x_blocks)
                states.append(flat_state)

                if self.detail_features and self.S > 0:
                    # Δ_s = x^{(s-1)} − x^{(s)}
                    delta_list = [
                        self.x_blocks[s - 1] - self.x_blocks[s] for s in range(1, self.S + 1)
                    ]
                    details.append(np.concatenate(delta_list))

        X_main = np.asarray(states, dtype=np.float32)  # [T-d, N]
        Y = targets[discard:]

        # feature map Φ
        if self.detail_features and self.S > 0:
            X_det = np.asarray(details, dtype=np.float32)  # same rows
            feats = np.concatenate(
                [X_main, X_det, np.ones((X_main.shape[0], 1), dtype=np.float32)], axis=1
            )
        else:
            feats = np.concatenate(
                [X_main, np.ones((X_main.shape[0], 1), dtype=np.float32)], axis=1
            )

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(feats, Y)
        self.W_out = reg.coef_.astype(np.float32)

    # ------------------------------------------------------------------ #
    #                       Autoregressive rollout                        #
    # ------------------------------------------------------------------ #
    def predict_autoregressive(
        self, initial_input: np.ndarray, n_steps: int
    ) -> np.ndarray:
        if self.W_out is None:
            raise RuntimeError("fit_readout() must be called first")

        d_in = initial_input.shape[0]
        d_out = self.W_out.shape[0]
        preds = np.empty((n_steps, d_out), dtype=np.float32)

        #self.reset_state()
        current_u = initial_input.astype(np.float32).copy()

        for t in range(n_steps):
            self._single_step(current_u)

            flat_state = np.concatenate(self.x_blocks)
            if self.detail_features and self.S > 0:
                delta_list = [
                    self.x_blocks[s - 1] - self.x_blocks[s] for s in range(1, self.S + 1)
                ]
                feat_vec = np.concatenate([flat_state, *delta_list, [1.0]])
            else:
                feat_vec = np.concatenate([flat_state, [1.0]])

            y_t = (self.W_out @ feat_vec).astype(np.float32)
            preds[t] = y_t
            current_u = y_t[:d_in]

        return preds

# Unweighted Erdős–Rényi random graph
from scipy import sparse

n0      = 300               # number of nodes
p_edge  = 4 / n0            # expected degree ≈ 4

rng  = np.random.default_rng(123)
rows = rng.choice(n0, size=int(p_edge * n0 * (n0 - 1) // 2))
cols = rng.choice(n0, size=rows.size)
mask = rows != cols         # avoid self-loops
rows, cols = rows[mask], cols[mask]


# Build upper triangle, then symmetrise
adj = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), shape=(n0, n0))
adj = adj + adj.T
adj[adj > 0] = 1.0  # Make unweighted (0/1)

# Convert to CSC for downstream sparse linear algebra
adj = adj.tocsc()


'''
# 4-nearest-neighbour ring lattice (simple and deterministic)
from scipy import sparse
n0  = 128
data, row, col = [], [], []
for i in range(n0):
    for k in (-2, -1, 1, 2):           # 4 neighbours
        j = (i + k) % n0
        row.append(i); col.append(j); data.append(1.0)

adj = sparse.csr_matrix((data, (row, col)), shape=(n0, n0))
'''

'''
# Using NetworkX for more elaborate topologies
import networkx as nx
from scipy import sparse

n0 = 128
G  = nx.random_geometric_graph(n0, radius=0.20, seed=123)
adj = nx.to_scipy_sparse_array(G, dtype=float, format="csr")
adj[adj > 0] = 1.0          # optional: binarise edge weights
'''

'\n# Using NetworkX for more elaborate topologies\nimport networkx as nx\nfrom scipy import sparse\n\nn0 = 128\nG  = nx.random_geometric_graph(n0, radius=0.20, seed=123)\nadj = nx.to_scipy_sparse_array(G, dtype=float, format="csr")\nadj[adj > 0] = 1.0          # optional: binarise edge weights\n'

In [5]:
grid = {
    "tau0": [0.05],
    "betas": [[0.5, 0.4]],
    "alphas": [[0.6, 0.7, 1.0]],
    "input_scale": [0.1],
    "ridge_alpha": [1e-8],
}

In [6]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    # Precompute param combinations
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # tqdm adds a progress bar for better visualization
    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores = []
        
        # Run all 20 seeds
        for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
            tmax = 250
            dt   = 0.02
            t_vals, lorenz_traj = generate_lorenz_data(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )
            
            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]
            
            # normalize
            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)
            
            T_data = len(lorenz_traj)
            for train_frac in [0.7, 0.75, 0.8]:
                train_end = int(train_frac*(T_data-1))
                train_input  = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end+1]
                test_input   = lorenz_traj[train_end:-1]
                test_target  = lorenz_traj[train_end+1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]
                for seed in np.arange(1, 6):
                    model = model_class(**params, adj=adj, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    T_VPT_s, _, _ = compute_valid_prediction_time(test_target, preds, t_vals, 0.4, 0.9, dt)
                    seed_scores.append(T_VPT_s)
        mean_score = float(np.mean(seed_scores))
        std_dev    = float(np.std(seed_scores))
        # is_stable  = std_dev < 1.5
        # status     = "Stable" if is_stable else "Unstable"
        
        # print(f"Params: {params} → Avg T_VPT={mean_score:.3f}, "
        #       f"Std Dev={std_dev:.3f} → {status}")

        results.append({
            "params":      params,
            "seed_scores": seed_scores,
            "mean_T_VPT":  mean_score,
            "std_dev":     std_dev,
            # "stable":      is_stable
        })

    # Save results
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")
    
    return results

In [7]:
run_grid_search(DWMSR3D, grid, "DWMSR", output_path="dwmsr.json")


== Initial grid search for DWMSR with 1 combinations ==


Grid Search:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\shrey\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\shrey\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\shrey\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\shrey\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\shrey\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\sparse\


All results saved to `dwmsr.json`


[{'params': {'tau0': 0.05,
   'betas': [0.5, 0.4],
   'alphas': [0.6, 0.7, 1.0],
   'input_scale': 0.1,
   'ridge_alpha': 1e-08},
  'seed_scores': [np.float64(8.64),
   np.float64(8.520000000000003),
   np.float64(8.480000000000004),
   np.float64(8.5),
   np.float64(12.32),
   np.float64(8.740000000000002),
   np.float64(7.18),
   np.float64(8.68),
   np.float64(7.280000000000001),
   np.float64(11.82),
   np.float64(7.300000000000004),
   np.float64(7.480000000000004),
   np.float64(7.359999999999999),
   np.float64(11.939999999999998),
   np.float64(9.68),
   np.float64(8.880000000000003),
   np.float64(3.8400000000000034),
   np.float64(8.740000000000002),
   np.float64(8.740000000000002),
   np.float64(8.800000000000004),
   np.float64(8.740000000000002),
   np.float64(9.480000000000004),
   np.float64(9.420000000000002),
   np.float64(12.439999999999998),
   np.float64(10.18),
   np.float64(6.800000000000004),
   np.float64(11.920000000000002),
   np.float64(6.719999999999999),
 